O KNN É um dos primeiros algoritmos aprendido no estudo de ML e é utilizado em problemas de classificação, ou sejam, tenta prever os resultados em uma saída discreta, em outras palavras, ele quer mapear variáveis ​​de entrada em categorias distintas.

o KNN tenta classificar cada amostra de um conjunto de dados avaliando sua distância em relação aos vizinhos mais próximos. Se os vizinhos mais próximos forem majoritariamente de uma classe, a amostra em questão será classificada nesta categoria.

In [1]:
import pandas as pd
# import dummy classifier
from sklearn.dummy import DummyClassifier
# import train_test_split
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,78,1,1,3,2.312,4
1,42,0,2,2,3.560,3
2,31,0,1,3,2.926,3
3,34,1,2,3,1.924,3
4,27,1,2,3,1.807,3


In [5]:
x_train, x_val, y_train, y_val = train_test_split(data.drop('Drug', axis=1), data['Drug'], test_size=0.2, random_state=0)

In [6]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train, y_train)

dummy_clf.score(x_val, y_val)

0.485625

In [7]:
#knn algorithm
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train, y_train)

knn_clf.score(x_val, y_val)

0.93

In [8]:
#import min max scaler, one hot encoder, and column transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# pode deixar dessa forma, se tiver mt ruim a gente muda
var_to_be_used = ['BP', 'Cholesterol', 'Na_to_K']
numerical_features = ['BP', 'Cholesterol', 'Na_to_K']
categorical_features = []

# create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_train = preprocessor.fit_transform(x_train)
x_val = preprocessor.transform(x_val)

In [9]:
# import ransearchcv
from sklearn.model_selection import RandomizedSearchCV

# define the parameters
params = {
    'weights':['uniform', 'distance'],
    'n_neighbors': [3, 4, 5, 6, 7, 8],
    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
}
# create the model
knn_clf = KNeighborsClassifier()

# create the random search cv
random_search = RandomizedSearchCV(knn_clf, param_distributions = params, n_iter = 5, scoring = 'roc_auc', n_jobs = -1, cv = 5, verbose = 3)

# fit the model
random_search.fit(x_train, y_train)

random_search.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\Ianme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


{'weights': 'uniform', 'n_neighbors': 7, 'algorithm': 'auto'}

In [10]:
# train knn with the best params
knn_clf = KNeighborsClassifier(**random_search.best_params_)
knn_clf.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [11]:
knn_clf.score(x_val, y_val)

0.59

In [12]:
data_test = pd.read_csv('test.csv')

In [14]:
data_test['Drug'].shape

(2000,)

In [15]:
x_test = preprocessor.transform(data_test)
y_test = knn_clf.predict(x_test)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(data_test['Drug'], y_test)

0.579